In [3]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.types import Command
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage

# 1) Define the shape of our state
class State(TypedDict):
    messages: list

# 2) Node: ask for name
def ask_name(state: State):
    return {"messages": [AIMessage(content="What’s your name?")]}

# 3) Node: pass-through human node
def human_pass(state: State):
    # after resume, state["messages"] already contains your HumanMessage
    return {"messages": state["messages"]}

# 4) Node: echo greeting
def greet(state: State):
    name_msg = state["messages"][-1]   # this is the HumanMessage
    return {"messages": [AIMessage(content=f"Hello, {name_msg.content}!")]}

# 5) Build the graph
builder = StateGraph(State)
builder.add_node("ask",   ask_name)
builder.add_node("human", human_pass)
builder.add_node("greet", greet)
builder.add_edge("ask",   "human")
builder.add_edge("human", "greet")
builder.set_entry_point("ask")

# 6) Compile with interrupt on the human node
graph = builder.compile(
    checkpointer=MemorySaver(),
    interrupt_before=["human"],
)

# 7) Prepare a config with a thread_id so MemorySaver will work
config = {"thread_id": "human_loop_example"}

# 8) First invocation — runs ask_name, then stops at "human"
result = graph.invoke({}, config=config)
print("Interrupted?", "__interrupt__" in result)   # → True

# 9) Resume by injecting your HumanMessage via Command
resume = Command(resume={
    "action": "update",
    "data": {"content": "Alice"}    # your name here
})
final = graph.invoke(resume, config=config)

# 10) Inspect the bot’s reply
print(final["messages"][-1].content)  # → "Hello, Alice!"


Interrupted? False
Hello, What’s your name?!


In [5]:
# … everything else is exactly as before …

# 7) First invocation — tell it to stream updates so it stops at your human node
result = graph.invoke(
    {}, 
    config=config, 
    stream_mode="updates"       # ← this line makes it yield an interrupt
)
print("Interrupted?", "__interrupt__" in result)   # → True



Interrupted? False


In [6]:
# 8) Now it’s paused at "human". Resume as before:
resume = Command(resume={
    "action": "update",
    "data": {"content": "Alice"}
})
final = graph.invoke(resume, config=config)
print(final["messages"][-1].content)  # → "Hello, Alice!

Hello, What’s your name?!


In [7]:
# … after your compile() and config setup …

# 1) Invoke with an explicit interrupt_before
result = graph.invoke(
    {}, 
    config=config, 
    interrupt_before=["human"]
)

print("Interrupted?", "__interrupt__" in result)   # → True
# result["__interrupt__"] will tell you it’s waiting at "human"

# 2) Now resume as before:
resume = Command(resume={
    "action": "update",
    "data": {"content": "Alice"}    # your human input
})
final = graph.invoke(resume, config=config)

print(final["messages"][-1].content)  # → "Hello, Alice!"


Interrupted? False
Hello, What’s your name?!


In [9]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.types import Command
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage

# 1) Define your state
class State(TypedDict):
    messages: list

# 2) Asks for the user’s name
def ask_name(state: State):
    return {"messages": [AIMessage(content="What’s your name?")]}

# 3) A pass‑through “human” node
def human_pass(state: State):
    # after resume, state["messages"] includes the HumanMessage you injected
    return {"messages": state["messages"]}

# 4) Greets based on the human input
def greet(state: State):
    last: HumanMessage = state["messages"][-1]
    return {"messages": [AIMessage(content=f"Hello, {last.content}!")]}

# 5) Build your graph
builder = StateGraph(State)
builder.add_node("ask",   ask_name)
builder.add_node("human", human_pass)
builder.add_node("greet", greet)
builder.add_edge("ask",   "human")
builder.add_edge("human", "greet")
builder.set_entry_point("ask")

# 6) Compile — **no** interrupt configured here
graph = builder.compile(
    checkpointer=MemorySaver(),
)

# 7) Provide a thread_id so the MemorySaver can checkpoint
config = {"thread_id": "human_loop_example"}

# 8) **Invoke** with the interrupt directive **at** invoke-time
result = graph.invoke(
    {}, 
    config=config, 
    interrupt_before=["human"]
)
print("Interrupted?", "__interrupt__" in result)   # → True

# 9) Now resume by injecting your human reply
resume = Command(resume={
    "action": "update",
    "data": {"content": "Alice"}    # ← swap in any name you like
})
final = graph.invoke(resume, config=config)

# 10) Check the final greeting
print(final["messages"][-1].content)  # → "Hello, Alice!"


Interrupted? False
Hello, What’s your name?!


In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.types import Command
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage

# 1) Define your state
class State(TypedDict):
    messages: list

# 2) Node: ask for name
def ask_name(state: State):
    return {"messages": [AIMessage(content="What’s your name?")]}

# 3) Node: pass‑through human
def human_pass(state: State):
    return {"messages": state["messages"]}

# 4) Node: greeting
def greet(state: State):
    last: HumanMessage = state["messages"][-1]
    return {"messages": [AIMessage(content=f"Hello, {last.content}!")]}

# 5) Build the graph
builder = StateGraph(State)
builder.add_node("ask",   ask_name)
builder.add_node("human", human_pass)
builder.add_node("greet", greet)
builder.add_edge("ask",   "human")
builder.add_edge("human", "greet")
builder.set_entry_point("ask")

# 6) Compile (no interrupt flags here)
graph = builder.compile(checkpointer=MemorySaver())

# 7) Config for MemorySaver
config = {"thread_id": "human_loop_example"}

# 8) Invoke with streaming + interrupt_before
stream_result = graph.invoke(
    {},
    config=config,
    stream_mode="updates",
    interrupt_before=["human"]
)

# If it came back as a tuple, unpack the chunks list:
if isinstance(stream_result, tuple) and len(stream_result) >= 1:
    chunks = stream_result[0]
else:
    chunks = stream_result  # might already be a list

# Now find the interrupt chunk
interrupt_chunk = next(
    (c for c in chunks if isinstance(c, dict) and "__interrupt__" in c),
    None
)
if not interrupt_chunk:
    raise RuntimeError("No interrupt found—did the graph skip the human node?")
print("⏸ Paused at node:", interrupt_chunk["__interrupt__"]["node"])

# 9) Resume with a name
resume = Command(resume={
    "action": "update",
    "data": {"content": "Alice"}
})
resume_result = graph.invoke(resume, config=config, stream_mode="updates")

# Again unpack if needed
if isinstance(resume_result, tuple) and len(resume_result) >= 1:
    continue_chunks = resume_result[0]
else:
    continue_chunks = resume_result

# Pull out the final greeting
greet_chunk = next(
    (c for c in continue_chunks if isinstance(c, dict) and "messages" in c),
    None
)
print("🤖", greet_chunk["messages"][-1].content)  # → "Hello, Alice!"


TypeError: tuple indices must be integers or slices, not str

In [12]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.types import Command, interrupt
from langchain_core.messages import AIMessage, HumanMessage

class State(TypedDict):
    messages: list

def ask_name(state):
    return {"messages": [AIMessage(content="What’s your name?")]}

def human_node(state):
    # this will immediately return an interrupt at runtime
    return interrupt(
        state,
        prompt={"role":"assistant","content":"(waiting for human input)"}
    )

def greet(state):
    last: HumanMessage = state["messages"][-1]
    return {"messages": [AIMessage(content=f"Hello, {last.content}!")]}

builder = StateGraph(State)
builder.add_node("ask",   ask_name)
builder.add_node("human", human_node)
builder.add_node("greet", greet)
builder.add_edge("ask",   "human")
builder.add_edge("human", "greet")
builder.set_entry_point("ask")

graph = builder.compile()

# 1) Run once—you’ll get back a dict with "__interrupt__" in it
result = graph.invoke({}, interrupt_before=[])
print("Paused:", "__interrupt__" in result)

# 2) Now resume by sending your input
resume = Command(resume={"action":"update","data":{"content":"Alice"}})
final = graph.invoke(resume)
print(final["messages"][-1].content)  # → "Hello, Alice!"


TypeError: interrupt() got an unexpected keyword argument 'prompt'

In [13]:
from langgraph.graph import StateGraph
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict
from langchain_core.messages import AIMessage, HumanMessage


# 1. Define state schema
class State(TypedDict):
    messages: list


# 2. Ask the user
def ask_name(state: State):
    return {"messages": [AIMessage(content="What’s your name?")]}


# 3. Pause for human input using `interrupt`
def wait_for_human(state: State):
    return interrupt(state)  # triggers pause


# 4. Respond with greeting
def greet(state: State):
    human_msg = state["messages"][-1]  # last message is the human's response
    return {"messages": [AIMessage(content=f"Hello, {human_msg.content}!")]}


# 5. Build LangGraph
builder = StateGraph(State)
builder.add_node("ask", ask_name)
builder.add_node("wait", wait_for_human)
builder.add_node("greet", greet)

builder.add_edge("ask", "wait")
builder.add_edge("wait", "greet")
builder.set_entry_point("ask")

graph = builder.compile(checkpointer=MemorySaver())

# 6. Initial run (pauses at 'wait' via `interrupt`)
config = {"thread_id": "human_demo_123"}
result = graph.invoke({}, config=config)

# Should contain "__interrupt__"
print("Interrupted?", "__interrupt__" in result)
print(result["messages"][-1].content)  # Should be "What’s your name?"

# 7. Resume manually by injecting human response
resume_cmd = Command(resume={
    "action": "update",
    "data": {"content": "Alice"}  # Replace with any name
})
final_result = graph.invoke(resume_cmd, config=config)

# Final message
print("🤖", final_result["messages"][-1].content)  # → "Hello, Alice!"


Interrupted? True
What’s your name?
🤖 Hello, What’s your name?!


In [14]:
from langchain_core.messages import HumanMessage

resume_cmd = Command(resume={
    "action": "update",
    "data": HumanMessage(content="Alice")   # ✅ wrap properly
})


In [15]:
final_result = graph.invoke(resume_cmd, config=config)
print("🤖", final_result["messages"][-1].content)


🤖 Hello, What’s your name?!


In [1]:
# pip install langgraph langchain langchain-core langchain-tavily typing_extensions

import uuid
from typing import Annotated
from typing_extensions import TypedDict

from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import InMemorySaver

# 1) State with message history
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 2) LLM + tool that pauses for human input
llm = ChatOpenAI(model="gpt-4", temperature=0.7)

@tool
def human_assistance(query: str) -> str:
    """Request assistance/approval from a human."""
    resp = interrupt({"query": query})         # pause here
    return resp["data"]                         # resume fills this

tools = [human_assistance]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    msg = llm_with_tools.invoke(state["messages"])
    assert len(msg.tool_calls) <= 1             # avoid dup calls on resume
    return {"messages": [msg]}

# 3) Build graph
builder = StateGraph(State)
builder.add_node("chatbot", chatbot)
builder.add_node("tools", ToolNode(tools=tools))
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools", "chatbot")
builder.add_edge(START, "chatbot")

graph = builder.compile(checkpointer=InMemorySaver())
config = {"configurable": {"thread_id": "1"}}

# 4) Run – it will pause when tool is invoked
events = graph.stream({"messages": [{"role": "user", "content": "Please ask an expert for tips."}]},
                      config, stream_mode="values")
for _ in events:
    pass  # you'll receive an interrupt with {"query": ...}

# 5) Resume with human input
human_text = "Expert says: prefer LangGraph for reliable agents."
graph.stream(Command(resume={"data": human_text}), config, stream_mode="values")


ImportError: cannot import name 'ToolNode' from 'langgraph.prebuilt' (unknown location)

In [3]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import AIMessage, ToolMessage
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model

class State(TypedDict):
    messages: Annotated[list, add_messages]

@tool
def ping(x: int) -> str:
    """Return pong with x."""
    return f"pong:{x}"

tools = [ping]
llm = ChatOpenAI(model="gpt-4", temperature=0.7).bind_tools(tools)

def agent(state: State):
    ai = llm.invoke(state["messages"])
    out = [ai]
    # Manually execute any tool calls
    for tc in ai.tool_calls or []:
        func = {t.name: t for t in tools}[tc["name"]]
        result = func.invoke(tc["args"])
        out.append(ToolMessage(tool_call_id=tc["id"], content=str(result)))
    return {"messages": out}

g = StateGraph(State)
g.add_node("agent", agent)
g.add_edge(START, "agent")
g.add_edge("agent", END)
graph = g.compile()


NameError: name 'ChatOpenAI' is not defined

In [4]:
# pip install -U langgraph langchain-core

from typing import TypedDict, Annotated, Literal
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import interrupt

# ---------- State definition ----------
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]  # conversation transcript
    approved: Literal["yes", "no", "unknown"]  # human approval flag

# ---------- Nodes ----------
def plan(state: GraphState) -> GraphState:
    """Draft a plan based on the last user message."""
    last_user = next((m.content for m in reversed(state["messages"]) if isinstance(m, HumanMessage)), "")
    proposal = f"Plan: I will carry out the request -> {last_user}. Approve? (yes/no)"
    return {
        "messages": [AIMessage(content=proposal)]
    }

def need_approval(state: GraphState) -> GraphState:
    """
    Pause the graph and ask for human approval.
    The first time this runs, it interrupts. When resumed, it receives the human's answer.
    """
    # Ask the human. The value you pass to `resume(...)` later will be returned here.
    answer = interrupt({"question": "Approve the plan? Reply 'yes' or 'no'."})
    # normalize
    ans = str(answer).strip().lower()
    if ans not in ("yes", "no"):
        ans = "no"
    return {"approved": ans}

def execute(state: GraphState) -> GraphState:
    """Do something based on approval."""
    if state.get("approved", "unknown") != "yes":
        return {"messages": [AIMessage("Execution cancelled based on your decision.")]}
    return {"messages": [AIMessage("Executing the approved plan now ✅")]}

# ---------- Graph wiring ----------
builder = StateGraph(GraphState)
builder.add_node("plan", plan)
builder.add_node("need_approval", need_approval)
builder.add_node("execute", execute)

builder.add_edge(START, "plan")
builder.add_edge("plan", "need_approval")
builder.add_edge("need_approval", "execute")
builder.add_edge("execute", END)

# Checkpointer is required to resume the same thread
memory = MemorySaver()
app = builder.compile(checkpointer=memory)

# ---------- Example usage ----------
# Give every conversation a stable thread_id so you can resume it after an interrupt.
config = {"configurable": {"thread_id": "demo-thread-1"}}

# 1) Kick off the run. Use .stream so we can detect the interrupt event.
print("\n--- Starting run (expect an interrupt asking for approval) ---")
events = list(app.stream(
    {"messages": [HumanMessage("Book a table for two at 7pm and send me a confirmation.")]},
    config,
    stream_mode="values",   # yields state snapshots as it advances
))

# At this point, the graph has paused at `need_approval`.
# You can inspect `events` if you want, but the important step is to RESUME with human input.

# 2) Human responds out-of-band (e.g., UI form, CLI input). We'll hardcode "yes" here:
human_reply = "yes"

print("\n--- Resuming with human approval:", human_reply, "---")
# .resume sends the value back into the most recent interrupt for this thread
resumed_events = list(app.resume(human_reply, config, stream_mode="values"))

# 3) Final state / messages
final_state = resumed_events[-1] if resumed_events else {}
print("\n--- Final assistant messages ---")
for msg in final_state.get("messages", []):
    role = "User" if isinstance(msg, HumanMessage) else "Assistant"
    print(f"{role}: {msg.content}")



--- Starting run (expect an interrupt asking for approval) ---

--- Resuming with human approval: yes ---


AttributeError: 'CompiledStateGraph' object has no attribute 'resume'

In [7]:
# pip install -U langgraph langchain-core

from typing import TypedDict, Annotated, Literal
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver   # <-- use a checkpointer
from langgraph.types import interrupt, Command          # <-- Command is the key

# ---------- State ----------
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    approved: Literal["yes", "no", "unknown"]

# ---------- Nodes ----------
def plan(state: GraphState) -> GraphState:
    last_user = next((m.content for m in reversed(state["messages"]) if isinstance(m, HumanMessage)), "")
    proposal = f"Plan: I will carry out the request -> {last_user}. Approve? (yes/no)"
    return {"messages": [AIMessage(content=proposal)]}

def need_approval(state: GraphState) -> GraphState:
    answer = interrupt({"question": "Approve the plan? Reply 'yes' or 'no'."})
    ans = str(answer).strip().lower()
    if ans not in ("yes", "no"):
        ans = "no"
    return {"approved": ans}

def execute(state: GraphState) -> GraphState:
    if state.get("approved", "unknown") != "yes":
        return {"messages": [AIMessage("Execution cancelled based on your decision.")]}
    return {"messages": [AIMessage("Executing the approved plan now ✅")]}

# ---------- Graph wiring ----------
builder = StateGraph(GraphState)
builder.add_node("plan", plan)
builder.add_node("need_approval", need_approval)
builder.add_node("execute", execute)

builder.add_edge(START, "plan")
builder.add_edge("plan", "need_approval")
builder.add_edge("need_approval", "execute")
builder.add_edge("execute", END)

checkpointer = InMemorySaver()
app = builder.compile(checkpointer=checkpointer)

# ---------- Example run ----------
config = {"configurable": {"thread_id": "demo-thread-1"}}

print("\n--- Starting run (expect an interrupt asking for approval) ---")
events = list(app.stream(
    {"messages": [HumanMessage("Book a table for two at 7pm and send me a confirmation.")]},
    config,
    stream_mode="values",
))

# Simulate human approval gathered from your UI / CLI:
human_reply = input()

print("\n--- Resuming with human approval:", human_reply, "---")
# ✅ Correct way to resume:
resumed_events = list(app.stream(Command(resume=human_reply), config, stream_mode="values"))

final_state = resumed_events[-1] if resumed_events else {}
print("\n--- Final assistant messages ---")
for msg in final_state.get("messages", []):
    role = "User" if isinstance(msg, HumanMessage) else "Assistant"
    print(f"{role}: {msg.content}")



--- Starting run (expect an interrupt asking for approval) ---

--- Resuming with human approval: yes ---

--- Final assistant messages ---
User: Book a table for two at 7pm and send me a confirmation.
Assistant: Plan: I will carry out the request -> Book a table for two at 7pm and send me a confirmation.. Approve? (yes/no)
Assistant: Executing the approved plan now ✅
